In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import os
from skimage import measure
from scipy import interpolate
from pathlib import Path
import pandas as pd
from radiomicsutil.voi import find_contours, getMask, calc_vol, calc_mesh, smooth_mesh, plot_mesh, plot_mesh3d
from radiomicsutil.threed import ThreedMorphologyFeaturesExtractor
# 指定.npy檔案的路徑


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getLargestContour(img: np.array):
    img = onlyContours(img)
   # print(img)
    #print(dff)
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #width = 0
    #high = 0
    if not contours:
        return None 
    
    areas = [cv2.contourArea(cnt) for cnt in contours]
    max_idx = np.argmax(areas)
    largest_contour = contours[max_idx]
    
    
    mask = np.zeros((800, 800), dtype=np.uint8)
    #mask = cv2.drawContours(mask, [largest_contour], 0, 255, 10)
    cv2.drawContours(mask, [largest_contour], 0, 255, thickness=cv2.FILLED)
   
    #img =  cv2.bitwise_or(img, (np.zeros((800, 800), dtype=np.uint8)))
    return np.array(mask)
def onlyContours(img: np.array):
    rows, cols, els = img.shape
    bin = np.zeros((rows, cols), dtype='uint8')
    for i in range(rows):
        for j in range(cols):
            if (img[i,j,0] == img[i,j,1] == img[i,j,2] ):
                bin[i,j] = 0
            elif((img[i,j,2]-img[i,j,1]) > 50): 
                bin[i,j] = 255
            else: bin[i,j] = 0

    return bin

計算單相位3d資料

In [ ]:
# root_folder = 'C:/Users/Max/Desktop/肝癌手術電燒/radiomics 3D/HCCP'
root_folder = 'D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output3/check_img/'
results = []
# 获取根文件夹下的所有子文件夹名称
subfolder_names = [subfolder for subfolder in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, subfolder))]

# 遍历每个子文件夹
for subfolder_name in subfolder_names:
    try:
        subfolder_path = os.path.join(root_folder, subfolder_name)
    
        npy_file_path = os.path.join(subfolder_path, 'imgs.npy')
        image_data = np.load(npy_file_path)

        # 使用os.path.basename获取文件名部分
        file_name = os.path.basename(os.path.dirname(npy_file_path))
        print(file_name)
        # 使用字符串分割方法分割文件名
        parts = file_name.split('-')

        # 提取0003并存储到变量中
        if len(parts) >= 2:
            identifier = parts[1]

        # print(identifier)  # 输出变量identifier
        masks = list()
        for idx in range(len(image_data)):
            image = getLargestContour(image_data[idx])
            masks.append(image)
        slicefilename = f"{file_name}_slicelocations.pkl"
        slicefilepath = os.path.join(subfolder_path, slicefilename)
        # print(slicefilepath)
        with open(slicefilepath, "rb") as f:
            slice_locations = pickle.load(f)
        slice_locations = slice_locations[:len(masks)]
        sorted_indices = np.argsort(slice_locations)
        print(sorted_indices)
        slice_locations.sort()
        print(slice_locations)
        masks_sorted = [masks[i] for i in sorted_indices]

        pixelfilename = f"{file_name}_pixelspacing.pkl"
        pixelfilepath = os.path.join(subfolder_path, pixelfilename)
        with open(pixelfilepath, "rb") as f:
            case_pixel_spacings = pickle.load(f)
        px_spacing = case_pixel_spacings[0]
        # print(px_spacing)
        slice_thickness = 5.0
        # Create 3D grid for interpolation
        z = np.array(slice_locations)
        y = np.arange(0, masks[0].shape[0] * px_spacing, px_spacing)
        x = np.arange(0, masks[0].shape[1] * px_spacing, px_spacing)
        # Convert sorted masks to binary and stack along the z-axis
        masks_bin = np.array([(mask > 0).astype(int) for mask in masks_sorted])
        # Create 3D grid for the masks
        z_mask = np.arange(z.min(), z.max() + slice_thickness, slice_thickness)
        x_mask, y_mask, z_mask = np.meshgrid(x, y, z_mask, indexing='ij')
        # Create the interpolator function
        interpolator = interpolate.RegularGridInterpolator((x, y, z), np.moveaxis(masks_bin, 0, -1), method='nearest', bounds_error=False, fill_value=0)

        # Interpolate to fill missing slices
        vol = interpolator((x_mask, y_mask, z_mask))
        verts, faces, _, _ = measure.marching_cubes(vol, level=0.5)

        # Create a 3D plot
        # fig = plt.figure(figsize=(10, 10))
        # ax = fig.add_subplot(111, projection='3d')
        # # Fancy indexing: `verts[faces]` to generate a collection of triangles
        # ax.plot_trisurf(verts[:, 0], verts[:, 1], faces, verts[:, 2],
        #                 linewidth=0.2, antialiased=True, color='red', alpha=0.5)

        # plt.show()
        # Calculate the volume in cubic millimeters
        pixel_volume = px_spacing ** 2 * slice_thickness
        total_volume = np.sum(vol) * pixel_volume

        tumor_extractor = ThreedMorphologyFeaturesExtractor(vol)
        threed_morphology_p = tumor_extractor.all('p')
        threed_morphology_p['ISDnumber'] = pd.Series([identifier])
        results.append(threed_morphology_p)
    except Exception as e:
        print(f'处理文件{file_name}时发生错误：{str(e)}')
        continue  # 跳过当前文件的处理，继续处理下一个文件
final_results = pd.concat(results, ignore_index=True)
finalcsvpath = os.path.join(root_folder, 'p_phasevalue.csv')
final_results.to_csv(finalcsvpath,index=False)

批量計算多相位的3d資料

In [3]:
import os
import numpy as np
import pickle
import pandas as pd
from scipy import interpolate
from skimage import measure

# 定义一个函数来处理子文件夹
def process_subfolder(subfolder_path):
    # results = []

    # 获取子文件夹名称
    subfolder_name = os.path.basename(subfolder_path)

    try:
        npy_file_path = os.path.join(subfolder_path, 'imgs.npy')
        image_data = np.load(npy_file_path)
        file_name = os.path.basename(os.path.dirname(npy_file_path))
        print(file_name)
        # 使用字符串分割方法分割文件名
        parts = file_name.split('-')

        # 提取0003并存储到变量中
        if len(parts) >= 2:
            identifier = parts[1]
        masks = []
        for idx in range(len(image_data)):
            image = getLargestContour(image_data[idx])
            masks.append(image)

        slicefilename = f"{subfolder_name}_slicelocations.pkl"
        slicefilepath = os.path.join(subfolder_path, slicefilename)

        with open(slicefilepath, "rb") as f:
            slice_locations = pickle.load(f)
        slice_locations = np.unique(slice_locations)
        sorted_indices = np.argsort(slice_locations)
        slice_locations.sort()
        masks_sorted = [masks[i] for i in sorted_indices]

        pixelfilename = f"{subfolder_name}_pixelspacing.pkl"
        pixelfilepath = os.path.join(subfolder_path, pixelfilename)
        with open(pixelfilepath, "rb") as f:
            case_pixel_spacings = pickle.load(f)
        px_spacing = case_pixel_spacings[0]

        slice_thickness = 5.0

        # 创建插值器函数
        z = np.array(slice_locations)
        y = np.arange(0, masks[0].shape[0] * px_spacing, px_spacing)
        x = np.arange(0, masks[0].shape[1] * px_spacing, px_spacing)
        if len(x) != 800:
            x = np.arange(0, masks[0].shape[1] * px_spacing, px_spacing)[:-1]
        if len(y) != 800:
            y = np.arange(0, masks[0].shape[0] * px_spacing, px_spacing)[:-1]
        masks_bin = np.array([(mask > 0).astype(int) for mask in masks_sorted])
        z_mask = np.arange(z.min(), z.max() + slice_thickness, slice_thickness)
        x_mask, y_mask, z_mask = np.meshgrid(x, y, z_mask, indexing='ij')
        interpolator = interpolate.RegularGridInterpolator((x, y, z), np.moveaxis(masks_bin, 0, -1), method='nearest', bounds_error=False, fill_value=0)

        vol = interpolator((x_mask, y_mask, z_mask))
        verts, faces, _, _ = measure.marching_cubes(vol, level=0.5)

        pixel_volume = px_spacing ** 2 * slice_thickness
        total_volume = np.sum(vol) * pixel_volume

        tumor_extractor = ThreedMorphologyFeaturesExtractor(vol)
        threed_morphology_p = tumor_extractor.all('p')
        threed_morphology_p['ISDnumber'] = pd.Series([identifier])
        # results.append(threed_morphology_p)
        return threed_morphology_p
    except Exception as e:
        print(f'处理文件{subfolder_name}时发生错误：{str(e)}')
    
    # final_results = pd.concat(results, ignore_index=True)
    # new_path = os.path.dirname(subfolder_path)
    # last_folder_name = os.path.basename(new_path)
    # finalcsvpath = os.path.join(new_path, f'{last_folder_name}_phasevalue.csv')
    # final_results.to_csv(finalcsvpath, index=False)
    # print(finalcsvpath)

# 根文件夹
# root_folder = 'C:/Users/Max/Desktop/肝癌手術電燒/radiomics 3D/check_img'
root_folder = 'D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/'

subfolder_names = ['HCCA', 'HCCN', 'HCCP']

# 遍历每个子文件夹
for subfolder_name in subfolder_names:
    subfolder_path = os.path.join(root_folder, subfolder_name)
    print('subfolder_path:',subfolder_path)
    # 获取根文件夹下的所有子文件夹名称
    subfolder_filepaths = [subfolder_file for subfolder_file in os.listdir(subfolder_path) if os.path.isdir(os.path.join(subfolder_path, subfolder_file))]
    results = []
    # 遍历每个子文件夹
    for subfolder_filepath in subfolder_filepaths:
        
        datafile_path = os.path.join(subfolder_path, subfolder_filepath)
        print('datafile_path:',datafile_path)
        threed_morphology_p = process_subfolder(datafile_path)
        results.append(threed_morphology_p)
        final_results = pd.concat(results, ignore_index=True)
        new_path = os.path.dirname(datafile_path)
        last_folder_name = os.path.basename(new_path)
        finalcsvpath = os.path.join(new_path, f'{last_folder_name}_phasevalue_補充.csv')
        final_results.to_csv(finalcsvpath, index=False)
        print(finalcsvpath)



subfolder_path: D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCA
datafile_path: D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCA\ISD-0348
ISD-0348


C:\Users\Max\AppData\Local\Temp\ipykernel_12056\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


Use voxel to estimate volume...
D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCA\HCCA_phasevalue_補充.csv
datafile_path: D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCA\ISD-0658
ISD-0658


C:\Users\Max\AppData\Local\Temp\ipykernel_12056\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


Use voxel to estimate volume...
D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCA\HCCA_phasevalue_補充.csv
datafile_path: D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCA\ISD-1011
ISD-1011


...

Use voxel to estimate volume...
D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCP\HCCP_phasevalue_補充.csv
datafile_path: D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCP\ISD-1082
ISD-1082


C:\Users\Max\AppData\Local\Temp\ipykernel_12056\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


Use voxel to estimate volume...
D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCP\HCCP_phasevalue_補充.csv
datafile_path: D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCP\ISD-1120
ISD-1120


C:\Users\Max\AppData\Local\Temp\ipykernel_12056\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


Use voxel to estimate volume...
D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output4/check_img/HCCP\HCCP_phasevalue_補充.csv


從ISD-1xxx開始處理

In [ ]:
import os
import numpy as np
import pickle
import pandas as pd
from scipy import interpolate
from skimage import measure

# 定义一个函数来处理子文件夹
def process_subfolder(subfolder_path):
    # results = []

    # 获取子文件夹名称
    subfolder_name = os.path.basename(subfolder_path)

    try:
        npy_file_path = os.path.join(subfolder_path, 'imgs.npy')
        image_data = np.load(npy_file_path)
        file_name = os.path.basename(os.path.dirname(npy_file_path))
        print(file_name)
        # 使用字符串分割方法分割文件名
        parts = file_name.split('-')

        # 提取0003并存储到变量中
        if len(parts) >= 2:
            identifier = parts[1]
        masks = []
        for idx in range(len(image_data)):
            image = getLargestContour(image_data[idx])
            masks.append(image)

        slicefilename = f"{subfolder_name}_slicelocations.pkl"
        slicefilepath = os.path.join(subfolder_path, slicefilename)

        with open(slicefilepath, "rb") as f:
            slice_locations = pickle.load(f)
        slice_locations = slice_locations[:len(masks)]
        sorted_indices = np.argsort(slice_locations)
        slice_locations.sort()
        masks_sorted = [masks[i] for i in sorted_indices]

        pixelfilename = f"{subfolder_name}_pixelspacing.pkl"
        pixelfilepath = os.path.join(subfolder_path, pixelfilename)
        with open(pixelfilepath, "rb") as f:
            case_pixel_spacings = pickle.load(f)
        px_spacing = case_pixel_spacings[0]

        slice_thickness = 5.0

        # 创建插值器函数
        z = np.array(slice_locations)
        y = np.arange(0, masks[0].shape[0] * px_spacing, px_spacing)
        x = np.arange(0, masks[0].shape[1] * px_spacing, px_spacing)
        masks_bin = np.array([(mask > 0).astype(int) for mask in masks_sorted])
        z_mask = np.arange(z.min(), z.max() + slice_thickness, slice_thickness)
        x_mask, y_mask, z_mask = np.meshgrid(x, y, z_mask, indexing='ij')
        interpolator = interpolate.RegularGridInterpolator((x, y, z), np.moveaxis(masks_bin, 0, -1), method='nearest', bounds_error=False, fill_value=0)

        vol = interpolator((x_mask, y_mask, z_mask))
        verts, faces, _, _ = measure.marching_cubes(vol, level=0.5)

        pixel_volume = px_spacing ** 2 * slice_thickness
        total_volume = np.sum(vol) * pixel_volume

        tumor_extractor = ThreedMorphologyFeaturesExtractor(vol)
        threed_morphology_p = tumor_extractor.all('p')
        threed_morphology_p['ISDnumber'] = pd.Series([identifier])
        # results.append(threed_morphology_p)
        return threed_morphology_p
    except Exception as e:
        print(f'处理文件{subfolder_name}时发生错误：{str(e)}')
    
    # final_results = pd.concat(results, ignore_index=True)
    # new_path = os.path.dirname(subfolder_path)
    # last_folder_name = os.path.basename(new_path)
    # finalcsvpath = os.path.join(new_path, f'{last_folder_name}_phasevalue.csv')
    # final_results.to_csv(finalcsvpath, index=False)
    # print(finalcsvpath)

# 根文件夹
# root_folder = 'C:/Users/Max/Desktop/肝癌手術電燒/radiomics 3D/check_img'
root_folder = 'D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output3/check_img/'

# 定义要处理的文件夹名称顺序
subfolder_order = ['HCCA', 'HCCN', 'HCCP']

# 存储满足条件的子文件夹路径的列表
subfolder_paths_to_process = []

# 遍历每个子文件夹名称
for subfolder_name in subfolder_order:
    subfolder_path = os.path.join(root_folder, subfolder_name)
    
    # 获取根文件夹下的所有子文件夹名称
    subfolder_names = os.listdir(subfolder_path)
    
    # 遍历每个子文件夹
    for subfolder_name_inner in subfolder_names:
        subfolder_path_inner = os.path.join(subfolder_path, subfolder_name_inner)
        
        # 检查子文件夹是否为 ISD-1xxx 格式
        if os.path.isdir(subfolder_path_inner) and subfolder_name_inner.startswith('ISD-1'):
            # print('subfolder_path_inner:', subfolder_path_inner)
            
            # 将满足条件的子文件夹路径添加到列表中
            subfolder_paths_to_process.append(subfolder_path_inner)

# 遍历满足条件的子文件夹路径
for subfolder_path_inner in subfolder_paths_to_process:
    print('Processing subfolder:', subfolder_path_inner)
    
    # 调用处理函数来处理当前子文件夹
    threed_morphology_p = process_subfolder(subfolder_path_inner)
    
    # 下面的部分是您的处理代码，处理当前子文件夹的数据
    # ...

    # 接下来的代码将 threed_morphology_p 存储为 CSV 文件
    results.append(threed_morphology_p)
    final_results = pd.concat(results, ignore_index=True)
    new_path = os.path.dirname(subfolder_path_inner)
    last_folder_name = os.path.basename(new_path)
    finalcsvpath = os.path.join(new_path, f'{last_folder_name}_phasevalue2.csv')
    final_results.to_csv(finalcsvpath, index=False)
    print(finalcsvpath)

測試

In [48]:
import traceback
def process_subfolder_try(subfolder_path):
    # results = []

    # 获取子文件夹名称
    subfolder_name = os.path.basename(subfolder_path)

    try:
        npy_file_path = os.path.join(subfolder_path, 'imgs.npy')
        image_data = np.load(npy_file_path)
        file_name = os.path.basename(os.path.dirname(npy_file_path))
        print(file_name)
        # 使用字符串分割方法分割文件名
        parts = file_name.split('-')

        # 提取0003并存储到变量中
        if len(parts) >= 2:
            identifier = parts[1]
        masks = []
        for idx in range(len(image_data)):
            image = getLargestContour(image_data[idx])
            masks.append(image)

        slicefilename = f"{subfolder_name}_slicelocations.pkl"
        slicefilepath = os.path.join(subfolder_path, slicefilename)

        with open(slicefilepath, "rb") as f:
            slice_locations = pickle.load(f)
        # slice_locations = slice_locations[:len(masks)]
        slice_locations = np.unique(slice_locations)
        sorted_indices = np.argsort(slice_locations)
        slice_locations.sort()
        masks_sorted = [masks[i] for i in sorted_indices]

        pixelfilename = f"{subfolder_name}_pixelspacing.pkl"
        pixelfilepath = os.path.join(subfolder_path, pixelfilename)
        with open(pixelfilepath, "rb") as f:
            case_pixel_spacings = pickle.load(f)
        px_spacing = case_pixel_spacings[0]

        slice_thickness = 5.0

        # 创建插值器函数
        # masks_bin = np.array([(mask > 0).astype(int) for mask in masks_sorted])
        # print(len(masks_bin))
        z = np.array(slice_locations)
        # if len(z) > len(masks_bin):
        #     z = z[:-1] 
        y = np.arange(0, masks[0].shape[0] * px_spacing, px_spacing)
        x = np.arange(0, masks[0].shape[1] * px_spacing, px_spacing)
        if len(x) != 800:
            x = np.arange(0, masks[0].shape[1] * px_spacing, px_spacing)[:-1]
        if len(y) != 800:
            y = np.arange(0, masks[0].shape[0] * px_spacing, px_spacing)[:-1]
        masks_bin = np.array([(mask > 0).astype(int) for mask in masks_sorted])
        z_mask = np.arange(z.min(), z.max() + slice_thickness, slice_thickness)
        x_mask, y_mask, z_mask = np.meshgrid(x, y, z_mask, indexing='ij')
        interpolator = interpolate.RegularGridInterpolator((x, y, z), np.moveaxis(masks_bin, 0, -1), method='nearest', bounds_error=False, fill_value=0)

        vol = interpolator((x_mask, y_mask, z_mask))
        verts, faces, _, _ = measure.marching_cubes(vol, level=0.5)

        pixel_volume = px_spacing ** 2 * slice_thickness
        total_volume = np.sum(vol) * pixel_volume

        tumor_extractor = ThreedMorphologyFeaturesExtractor(vol)
        threed_morphology_p = tumor_extractor.all('p')
        threed_morphology_p['ISDnumber'] = pd.Series([identifier])
        # results.append(threed_morphology_p)
        return threed_morphology_p
    except Exception as e:
        traceback.print_exc()
        print(f'处理文件{subfolder_name}时发生错误：{str(e)}')

In [35]:
root = 'D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output3/check_img/HCCA/ISD-0272'
threed_morphology_p = process_subfolder_try(root)

ISD-0272


C:\Users\Max\AppData\Local\Temp\ipykernel_14536\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


处理文件ISD-0272时发生错误：list index out of range


Traceback (most recent call last):
  File "C:\Users\Max\AppData\Local\Temp\ipykernel_14536\3680454540.py", line 32, in process_subfolder_try
    masks_sorted = [masks[i] for i in sorted_indices]
  File "C:\Users\Max\AppData\Local\Temp\ipykernel_14536\3680454540.py", line 32, in <listcomp>
    masks_sorted = [masks[i] for i in sorted_indices]
IndexError: list index out of range


In [51]:
import pickle

# 指定.pkl檔案的路徑
missing_path = './check_img/HCCP/missing_paths.pkl'
# 使用二進位模式打開.pkl檔案
with open(missing_path, 'rb') as file:
    # 使用pickle.load()方法讀取檔案內容
    subfolder_paths_to_process = pickle.load(file)
results = []
failcase = []
for subfolder_path_inner in subfolder_paths_to_process:
    # print('Processing subfolder:', subfolder_path_inner)
    try:
        threed_morphology_p = process_subfolder_try(subfolder_path_inner)
        
        # 下面的部分是您的处理代码，处理当前子文件夹的数据
        # ...

        # 接下来的代码将 threed_morphology_p 存储为 CSV 文件
        results.append(threed_morphology_p)
        failcase.append(subfolder_path_inner)
    except Exception as e:
        print(f'处理文件{file_name}时发生错误：{str(e)}')
        continue 

final_results = pd.concat(results, ignore_index=True)
new_path = os.path.dirname(subfolder_path_inner)
last_folder_name = os.path.basename(new_path)
finalcsvpath = os.path.join(new_path, f'{last_folder_name}_phasevalue3.csv')
final_results.to_csv(finalcsvpath, index=False)
print(finalcsvpath)

ISD-0380


C:\Users\Max\AppData\Local\Temp\ipykernel_14536\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


Use voxel to estimate volume...
ISD-0364


C:\Users\Max\AppData\Local\Temp\ipykernel_14536\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


Use voxel to estimate volume...
ISD-0272


...

Use voxel to estimate volume...
ISD-0224


C:\Users\Max\AppData\Local\Temp\ipykernel_14536\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):


Use voxel to estimate volume...
ISD-0483


C:\Users\Max\AppData\Local\Temp\ipykernel_14536\2204506995.py:29: RuntimeWarning: overflow encountered in scalar subtract
  elif((img[i,j,2]-img[i,j,1]) > 50):
Traceback (most recent call last):
  File "C:\Users\Max\AppData\Local\Temp\ipykernel_14536\1808561733.py", line 61, in process_subfolder_try
    verts, faces, _, _ = measure.marching_cubes(vol, level=0.5)
  File "c:\Users\Max\anaconda3\envs\OP_RFA\lib\site-packages\skimage\measure\_marching_cubes_lewiner.py", line 133, in marching_cubes
    return _marching_cubes_lewiner(volume, level, spacing,
  File "c:\Users\Max\anaconda3\envs\OP_RFA\lib\site-packages\skimage\measure\_marching_cubes_lewiner.py", line 166, in _marching_cubes_lewiner
    raise ValueError("Input array must be at least 2x2x2.")
ValueError: Input array must be at least 2x2x2.


处理文件ISD-0483时发生错误：Input array must be at least 2x2x2.
ISD-0212
Use voxel to estimate volume...
D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output3/check_img/HCCP\HCCP_phasevalue3.csv


In [43]:
print(failcase)

None


In [8]:
import pickle

# 指定.pkl檔案的路徑
file_path = 'D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output3/check_img/HCCA/ISD-0364/ISD-0364_pixelspacing.pkl'

try:
    # 使用二進位模式打開.pkl檔案
    with open(file_path, 'rb') as file:
        # 使用pickle.load()方法讀取檔案內容
        data = pickle.load(file)
        
        # 顯示讀取到的資料型別
        print(f'讀取到的資料型別：{type(data)}')
        
        # 如果需要，您可以進一步處理讀取到的資料
        # 例如，將其列印出來
        print(data)
        
except FileNotFoundError:
    print(f'找不到檔案：{file_path}')
except Exception as e:
    print(f'發生錯誤：{str(e)}')

讀取到的資料型別：<class 'numpy.ndarray'>
[0.673828 0.673828]


In [36]:
import pickle

# 指定.pkl檔案的路徑
file_path = 'D:/共用雲端硬碟/TVGH-Group/手術復發/TGcode/Output3/check_img/HCCA/ISD-0209/ISD-0209_slicelocations.pkl'

try:
    # 使用二進位模式打開.pkl檔案
    with open(file_path, 'rb') as file:
        # 使用pickle.load()方法讀取檔案內容
        data = pickle.load(file)
        
        # 顯示讀取到的資料型別
        print(f'讀取到的資料型別：{type(data)}')
        
        # 如果需要，您可以進一步處理讀取到的資料
        # 例如，將其列印出來
        print(data)
        
except FileNotFoundError:
    print(f'找不到檔案：{file_path}')
except Exception as e:
    print(f'發生錯誤：{str(e)}')

讀取到的資料型別：<class 'numpy.ndarray'>
[-153 -148 -143 -138 -133 -128 -123 -118 -113 -108 -103  -98  -93  -88
  -88  -83  -78]
